The evaluation metric of this competition,
label-weighted label-ranking average precision,
is only affected by ranking of prediction scores.

I tried to transform raw prediction scores according to ranking of predictions before averaging them.
This blending strategy greatly improved the evaluation score. (〜 0.881 -> 0.895)

My best two submissions (public score = 0.881) were obtained by ensembling and blending many convnet predictions.
The file submission_879.csv is from [[AutoML] [Inference] Audio Detection - SoliSet](https://www.kaggle.com/hypnotu/automl-inference-audio-detection-soliset?scriptVersionId=52918021).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

CLASS_N = 24
features = [f's{i}' for i in range(CLASS_N)]

filename0 = '/kaggle/input/rfcx-my-best-submissions/mybest_881_1.csv'
filename1 = '/kaggle/input/rfcx-my-best-submissions/mybest_881_2.csv'
filename2 = '/kaggle/input/ensembling-0-880-audio-detection-101/submission.csv'
filename3 = '/kaggle/input/rfcx-my-best-submissions/submission_879.csv'
filename4 = '/kaggle/input/rfcx-bagging-with-different-weights-0-879-score/submission.csv'
filename5 = '/kaggle/input/bagging-rainforest/submission.csv'

df0 = pd.read_csv(filename0).sort_values('recording_id')
df1 = pd.read_csv(filename1).sort_values('recording_id')
df2 = pd.read_csv(filename2).sort_values('recording_id')
df3 = pd.read_csv(filename3).sort_values('recording_id')
df4 = pd.read_csv(filename4).sort_values('recording_id')
df5 = pd.read_csv(filename5).sort_values('recording_id')

X0 = df0[features].values
X1 = df1[features].values
X2 = df2[features].values
X3 = df3[features].values
X4 = df4[features].values
X5 = df5[features].values

Xs  = [X0, X1, X2, X3, X4, X5]

Because my two submissions are highly correlated, I assigned relatively small weights for my submissions.

In [ ]:
W = [0.14, 0.14, 0.215, 0.17, 0.17, 0.165]
print(sum(W))

Raw prediction scores are transformed by ranking of prediction.
Adding nonlinear term improved the public evaluation score.

In [ ]:
POINT = 30 - np.arange(CLASS_N) + np.cumsum(np.cumsum(0.03 * np.ones((CLASS_N, ))))
print(POINT)
def score_to_score(x):
    i = np.argsort(x)[::-1]
    r = np.argsort(i)
    return POINT[r]

In [ ]:
plt.figure()
plt.plot(range(1, CLASS_N+1), POINT)
plt.scatter(range(1, CLASS_N+1), POINT)
plt.xlabel('rank')
plt.ylabel('score')
plt.xlim([0, CLASS_N+1])
plt.show()

In [ ]:
N = X0.shape[0]
pred_list = []
for row in range(N):
    pred_list.append(np.sum(np.stack([w * score_to_score(X[row, :]) for X, w in zip(Xs, W)], axis=0), axis=0))
pred = np.stack(pred_list, axis=0)

data = {'recording_id' : df0['recording_id'].values }
for i in range(CLASS_N):
    data[f's{i}'] = pred[:, i]
sub = pd.DataFrame(data)
sub.to_csv("submission.csv", index=False)